In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
import time
import re
import urllib
import json
import numpy as np
import threading
from tqdm import tqdm
import pickle
import cloudscraper

In [6]:
# with open(f"link_data_stack_overflow_python_1981285.pickle", "rb") as f:
#     all_link = pickle.load(f)

In [4]:
# with open(f"link_data_stack_overflow_python_0_200000.pickle", "wb") as f:
#     pickle.dump(all_link[:200000],f)

In [7]:
# with open(f"link_data_stack_overflow_python_200000_400000.pickle", "wb") as f:
#     pickle.dump(all_link[200000:400000],f)

In [8]:
# with open(f"link_data_stack_overflow_python_400000_600000.pickle", "wb") as f:
#     pickle.dump(all_link[400000:600000],f)

In [9]:
# with open(f"link_data_stack_overflow_python_600000_800000.pickle", "wb") as f:
#     pickle.dump(all_link[600000:800000],f)

In [10]:
# with open(f"link_data_stack_overflow_python_800000_1000000.pickle", "wb") as f:
#     pickle.dump(all_link[800000:1000000],f)

In [12]:
with open(f"../use_link_load_data_0_1000000/link_data_stack_overflow_python_0_200000.pickle", "rb") as f:
    save_link = pickle.load(f)

In [13]:
len(save_link)

200000

In [14]:
max_pages = len(save_link)
count_thread = 80
per_vm = 8
vm_number = 1
num_per_thread = max_pages//count_thread
page_last = max_pages%count_thread
all_thread = [None]*count_thread
code_question = [None]*count_thread
code_answer = [None]*count_thread
webs = "https://stackoverflow.com/"
hours = 10
minute = 30

In [ ]:
def save_code_question_answer_all_data(code_question,code_answer,start,stop,number_thread):
    try:
        scraper = cloudscraper.create_scraper(delay=10,browser="chrome")
        st = time.time()
        if (code_question[number_thread] or code_answer[number_thread]) != None:
            code_answer_in_link = code_answer[number_thread][:start]
            code_question_in_link = code_question[number_thread][:start]
        else:
            code_answer_in_link = []
            code_question_in_link = []

        replace_code = [['<code>',''],['</code>',''],['&lt;','<'],['&gt;','>'],['&amp;','&'],['&quot;','"'],['&apos;',"'"],['>>> ',''],['... ',''],['...','']]
        count = start
        for question_link in save_link[start:stop]:
            if (time.time() - st) > (hours*60 + minute)*60:
                print(f"-->threading : {number_thread+1}\nlink : {count} finish with time stop hours : {hours} minute : {minute}\n{'*'*20}")
                return 0
            url_page = webs + question_link
            #------------------------------------
            while True:
                while True:
                    res_page = scraper.get(url_page)
                    if res_page.ok:
                        #print(f"Thread : {number_thread} page open")
                        break
                html_page = res_page.text
                soup_page = BeautifulSoup(html_page ,features="html.parser")
                #------------------------------------
                if len(soup_page.find_all('a',{'class':'s-pagination--item'}))/2 > 1:
                    last_page = soup_page.find_all('a',{'class':'s-pagination--item'})[-2].text
                else :last_page = 1
                code_question_reg = [str(soup_page.find_all('a',{'class':'question-hyperlink'})[0].text)]
                if soup_page.find_all('a',{'class':'question-hyperlink'})[0].text != None:
                    code_question_in_link.append(code_question_reg)
                    break
            answer = []
                #------------------------------------
            for page_in_question_link in range(int(last_page)):
                url_page = webs + question_link + f"?page={page_in_question_link+1}&tab=scoredesc#tab-top"
                while True:
                    while True:
                        res_page = scraper.get(url_page)
                        if res_page.ok:
                            #print(f"Thread : {number_thread} page open")
                            break
                    html_page = res_page.text
                    soup_page = BeautifulSoup(html_page ,features="html.parser")

                    answer_s = soup_page.find_all('div',{'id':'answers'})
                    # answer = soup_page.find_all('div',{'class':'s-prose'},{'class':'js-post-body'},{'id':'answers'})
                    # for i_answer in answer:
                    #     for i_code in i_answer.find_all('code'):
                    #         if len(str(i_code)) > 50:
                    #             i_code = str(i_code)
                    #             for rep in replace_code:
                    #                 i_code = i_code.replace(rep[0],rep[1])
                    #             # code_answer.append((((str(i_code).replace('<code>','')).replace('</code>','')).replace('&gt;&gt;&gt; ','')).replace('... ',''))
                    #             code_answer.append(i_code)
                    if answer_s != None:
                        answer.append(answer_s[0])#*************************

            code_answer_in_link.append(answer)#**************************
            count += 1
            if count%50==0:
                print(f"threading : {number_thread+1}\nlink : {count}\nprogress : {((count-start+1)/(stop-start))*100:.2f}%\n{'*'*20}")
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        print(f"-->threading : {number_thread+1}\nlink : {count} finish\n{'*'*20}")
    except:
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        print(f"-->threading : {number_thread+1}\nlink : {count} loop_interrup_finish\n{'*'*20}")
        save_code_question_answer_all_data(code_question,code_answer,count,stop,number_thread)





In [ ]:
li = (page_last - 1)
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    if i < page_last:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(all_link,all_status_link,i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i))
        print(i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i)
    elif page_last == 0:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(all_link,all_status_link,i*num_per_thread,(i+1)*num_per_thread,i))
        print(i*num_per_thread,(i+1)*num_per_thread,i)
    else:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(all_link,all_status_link,i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i))
        print(i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i)

In [ ]:
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].start()
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].join()